### Data Preprocessing (TF-IDF) and Train/Test split
---

**Goal:** To pre-process data with TF-IDF and split into train and test sets.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/MyDrive/Colab/github/XMTC")

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline
import csv

Read in tsv we prepared from previous notebook.

In [24]:
df = pd.read_csv('Task4preprocessed/netflix.tsv', sep='\t', index_col=0)
df.rename(columns={'description': 'plot_long'}, inplace=True)
columns_to_drop = ['country', 'date_added', 'release_year', 'duration', 'rating']
df.drop(columns=columns_to_drop, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,title,listed_in,plot_long,target_ages
0,Dick Johnson Is Dead,Documentaries,"As her father nears the end of his life, filmm...",Teens
1,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Adults
2,Ganglands,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Adults
3,Jailbirds New Orleans,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Adults
4,Kota Factory,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Adults
5,Midnight Mass,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,Adults
6,My Little Pony: A New Generation,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,Older Kids
7,Sankofa,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",Adults
8,The Great British Baking Show,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,Teens
9,The Starling,"Comedies, Dramas",A woman adjusting to life after a loss contend...,Teens


In [25]:
df = df.iloc[:, :4]
df.head(10)

,title,listed_in,plot_long,target_ages
0,Dick Johnson Is Dead,Documentaries,"As her father nears the end of his life, filmm...",Teens
1,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Adults
2,Ganglands,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Adults
3,Jailbirds New Orleans,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Adults
4,Kota Factory,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Adults
5,Midnight Mass,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,Adults
6,My Little Pony: A New Generation,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,Older Kids
7,Sankofa,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",Adults
8,The Great British Baking Show,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,Teens
9,The Starling,"Comedies, Dramas",A woman adjusting to life after a loss contend...,Teens


In [26]:
cols = list(df.columns.values)

In order to differentiate my features from the potentially thousands of new features created by TF-IDF, I will add `g_` to the beginning of the genre columns, and `f_` to the beginning of the feature columns.

In [27]:
genre_cols = cols[-1]
genre_cols = ['g_'+ genre_cols]
print(len(genre_cols))
print(genre_cols)

1
['g_target_ages']


In [28]:
feature_cols = ['f_'+f for f in cols[:3]]
#feature_cols = ['f_'+cols[0], 'f_'+cols[2]]
feature_cols

['f_title', 'f_listed_in', 'f_plot_long']

In [29]:
#cols = feature_cols + genre_cols + ['plot_long']
cols = feature_cols + genre_cols

In [30]:
print(cols)

['f_title', 'f_listed_in', 'f_plot_long', 'g_target_ages']


Columns have been renamed. Let's apply this to our dataset

In [31]:
df.columns = cols

---

**Split Data**

Split the data into our X and y where X are the independent variables (everything except the genres), and y holds our target variables (just the genres).

In [32]:
# X (movie info and plot) and y (genres)
X = df.drop(genre_cols, axis=1)
y = df[genre_cols]

In [33]:
X

,f_title,f_listed_in,f_plot_long
0,Dick Johnson Is Dead,Documentaries,"As her father nears the end of his life, filmm..."
1,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,Ganglands,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...
8789,Zodiac,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8790,Zombie Dumb,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8791,Zombieland,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8792,Zoom,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


---

**Pre-processing data (stem / lemmatize)**

Below are a series of functions to tokenize, stem, lemmatize and untokenize our text. All of these steps are need to 'clean' the text.

In [34]:
import numpy as np
import pandas as pd

# NLTK Packages
import nltk
# nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [35]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Croya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Croya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Croya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
from nltk.tokenize import word_tokenize
def tokenizer(text):
    """
    Tokenizes the document
    """
    return word_tokenize(text)


from nltk.corpus import stopwords

#Load up our stop words
stop_words = stopwords.words('english')
#Adds stuff to our stop words list
stop_words.extend(['.',','])


## This function can improve, simplify. Look into Text Data Lecture
def remove_stopwords(list_of_tokens):
    """
    Removes stopwords
    """

    cleaned_tokens = []

    for token in list_of_tokens:
        if token in stop_words: continue
        cleaned_tokens.append(token)

    return cleaned_tokens


from nltk.stem import PorterStemmer

def stemmer(list_of_tokens):
    '''
    Takes in an input which is a list of tokens, and spits out a list of stemmed tokens.
    '''

    stemmed_tokens_list = []

    for i in list_of_tokens:

        token = PorterStemmer().stem(i)
        stemmed_tokens_list.append(token)

    return stemmed_tokens_list


from nltk.stem import WordNetLemmatizer

def lemmatizer(list_of_tokens):

    lemmatized_tokens_list = []

    for i in list_of_tokens:
        token = WordNetLemmatizer().lemmatize(i)
        lemmatized_tokens_list.append(token)

    return lemmatized_tokens_list


def the_untokenizer(token_list):
        '''
        Returns all the tokenized words in the list to one string.
        Used after the pre processing, such as removing stopwords, and lemmatizing.
        '''
        return " ".join(token_list)

def cleaning_our_texts(text):

    cleaned_texts = []
    num_texts = len(text)
    #for i in range(len(text)):
        # Check if the element is a string; if not, convert it to a string
        #that_string = str(text[i])

    for i in text.index:
        that_string = text[i]

        tokenized_list = tokenizer(that_string)
        removed_stopwords = remove_stopwords(tokenized_list)
        stemmed_words = stemmer(removed_stopwords)
        lemmatized_words = lemmatizer(stemmed_words)
        back_to_string = the_untokenizer(lemmatized_words)

        cleaned_texts.append(back_to_string)

    return cleaned_texts

In [37]:
%%time
X['plot_clean'] = cleaning_our_texts(X['f_plot_long'])

CPU times: total: 4.89 s
Wall time: 5.8 s


In [38]:
#X = X.drop(['plot_long'],axis=1)
X = X.drop(['f_plot_long'],axis=1)
X

,f_title,f_listed_in,plot_clean
0,Dick Johnson Is Dead,Documentaries,a father near end life filmmak kirsten johnson...
1,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries",after cross path parti cape town teen set prov...
2,Ganglands,"Crime TV Shows, International TV Shows, TV Act...",to protect famili power drug lord skill thief ...
3,Jailbirds New Orleans,"Docuseries, Reality TV",feud flirtat toilet talk go among incarcer wom...
4,Kota Factory,"International TV Shows, Romantic TV Shows, TV ...",in citi coach center known train india ’ fines...
...,...,...,...
8789,Zodiac,"Cult Movies, Dramas, Thrillers",a polit cartoonist crime report pair cop inves...
8790,Zombie Dumb,"Kids' TV, Korean TV Shows, TV Comedies",while live alon spooki town young girl befrien...
8791,Zombieland,"Comedies, Horror Movies",look surviv world taken zombi dorki colleg stu...
8792,Zoom,"Children & Family Movies, Comedies",drag civilian life former superhero must train...


In [39]:
X['title_clean'] = cleaning_our_texts(X['f_title'])
X = X.drop(['f_title'],axis=1)
X

,f_listed_in,plot_clean,title_clean
0,Documentaries,a father near end life filmmak kirsten johnson...,dick johnson is dead
1,"International TV Shows, TV Dramas, TV Mysteries",after cross path parti cape town teen set prov...,blood & water
2,"Crime TV Shows, International TV Shows, TV Act...",to protect famili power drug lord skill thief ...,gangland
3,"Docuseries, Reality TV",feud flirtat toilet talk go among incarcer wom...,jailbird new orlean
4,"International TV Shows, Romantic TV Shows, TV ...",in citi coach center known train india ’ fines...,kota factori
...,...,...,...
8789,"Cult Movies, Dramas, Thrillers",a polit cartoonist crime report pair cop inves...,zodiac
8790,"Kids' TV, Korean TV Shows, TV Comedies",while live alon spooki town young girl befrien...,zombi dumb
8791,"Comedies, Horror Movies",look surviv world taken zombi dorki colleg stu...,zombieland
8792,"Children & Family Movies, Comedies",drag civilian life former superhero must train...,zoom


In [40]:
X['list_clean'] = cleaning_our_texts(X['f_listed_in'])
X = X.drop(['f_listed_in'],axis=1)
X

,plot_clean,title_clean,list_clean
0,a father near end life filmmak kirsten johnson...,dick johnson is dead,documentari
1,after cross path parti cape town teen set prov...,blood & water,intern tv show tv drama tv mysteri
2,to protect famili power drug lord skill thief ...,gangland,crime tv show intern tv show tv action & adventur
3,feud flirtat toilet talk go among incarcer wom...,jailbird new orlean,docuseri realiti tv
4,in citi coach center known train india ’ fines...,kota factori,intern tv show romant tv show tv comedi
...,...,...,...
8789,a polit cartoonist crime report pair cop inves...,zodiac,cult movi drama thriller
8790,while live alon spooki town young girl befrien...,zombi dumb,kid ' tv korean tv show tv comedi
8791,look surviv world taken zombi dorki colleg stu...,zombieland,comedi horror movi
8792,drag civilian life former superhero must train...,zoom,child & famili movi comedi


Next to vectorize the clean text.

---

But first, SPLIT data now for testing later and to avoid leakage.

In [41]:
from sklearn.model_selection import train_test_split
# Split with 25% test size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 123)

In [42]:
# NLP preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the Vectorizer
# min_df at 20 - token should appear in at least 20 documents to be counted
tfidf = TfidfVectorizer(min_df=20) #, ngram_range=(1,3))

# Fit the vectorizer to the training data
tfidf.fit(X_train['plot_clean'])

# Transform both train and test sets
X_train_tfidf = tfidf.transform(X_train['plot_clean'])
X_test_tfidf = tfidf.transform(X_test['plot_clean'])

# pickle tfidf model for later use
import joblib
joblib.dump(tfidf, '../Rating_Predict/models/netflix_tfidf_min20.pkl')

['../Rating_Predict/models/netflix_tfidf_min20.pkl']

In [43]:
X_train_tfidf

<6595x1041 sparse matrix of type '<class 'numpy.float64'>'
	with 64738 stored elements in Compressed Sparse Row format>

In [44]:
X_test_tfidf

<2199x1041 sparse matrix of type '<class 'numpy.float64'>'
	with 21261 stored elements in Compressed Sparse Row format>

---

Random 30 words as a sample.

In [45]:
feature_names = tfidf.get_feature_names_out()

pd.DataFrame(feature_names).sample(30)

#pd.DataFrame(tfidf.get_feature_names()).sample(30)

,0
7,30
1005,warrior
464,hunt
928,threaten
647,offici
116,bond
1021,win
491,intim
852,south
482,influenc


Change TD-IDF sparse matrix (for both train and test) back to dataframe format.

In [46]:
# Create new dataframe with the index of X_train, columns as the content of X_train_transformed token/feature names
feature_names = tfidf.get_feature_names_out()

# Create a DataFrame with the transformed data
X_train_transformed_df = pd.DataFrame(X_train_tfidf.toarray(), index=X_train.index, columns=feature_names)
print(X_train_transformed_df)

#X_train_transformed_df = pd.DataFrame(X_train_tfidf.toarray(), index=X_train.index, columns=tfidf.get_feature_names())
#X_train_transformed_df

      000   10   11  1960  1970  1980   20   30  abandon  abil  abus  accept  \
4537  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
1500  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
7023  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
6300  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
3920  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
...   ...  ...  ...   ...   ...   ...  ...  ...      ...   ...   ...     ...   
7382  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
7763  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
5218  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
1346  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
3582  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   

      accid  accident  account  accus  

In [47]:
# Create new dataframe with the index of X_test, columns as the content of X_test_transformed token/feature names
feature_names = tfidf.get_feature_names_out()

# Create a DataFrame with the transformed data
X_test_transformed_df = pd.DataFrame(X_test_tfidf.toarray(), index=X_test.index, columns=feature_names)
print(X_test_transformed_df)

#X_test_transformed_df = pd.DataFrame(X_test_tfidf.toarray(), index=X_test.index, columns=tfidf.get_feature_names())
#X_test_transformed_df

      000   10   11  1960  1970  1980   20   30  abandon  abil  abus  accept  \
5072  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
5565  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
506   0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
6146  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
2988  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
...   ...  ...  ...   ...   ...   ...  ...  ...      ...   ...   ...     ...   
4764  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
4654  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
8117  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
1984  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
963   0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   

      accid  accident  account  accus  

Finally, add back on the numeric features for our complete `X_train`.

In [48]:
# Create final X_train dataframe which has been transformed, and only has numeric columns.
X_train_transformed_df_full = X_train.join(X_train_transformed_df, on=X_train_transformed_df.index)
X_train_transformed_df_full.shape

(6595, 1044)

In [49]:
X_test_transformed_df_full = X_test.join(X_test_transformed_df, on=X_test_transformed_df.index)
X_test_transformed_df_full.shape

(2199, 1044)

Last bit to do is put the `y` values back on to train and test sets and export to be imported in the modeling workbook.

Once joined there are some multi-level index issues to work out.

In [50]:
train_df_export = X_train_transformed_df_full.drop(['plot_clean'], axis=1).copy()
train_df_export = train_df_export.merge(y_train, on=train_df_export.index)
train_df_export = train_df_export.set_index(['key_0'])
train_df_export.reset_index(drop=True, inplace=True)

#del train_df_export.index.name
train_df_export

,title_clean,list_clean,000,10,11,1960,1970,1980,20,30,abandon,abil,abus,accept,accid,accident,account,accus,across,act,action,activist,actor,actress,adapt,addict,adopt,adult,adventur,affair,africa,african,after,ag,age,agent,ago,agre,air,alien,aliv,all,alli,alon,along,also,alway,ambiti,america,american,amid,among,an,ancient,and,angel,anim,anoth,answer,anyth,apart,appear,arm,armi,around,arrang,arriv,art,artist,ask,aspir,assassin,assist,at,athlet,attack,attempt,attend,attract,author,aveng,avoid,await,awaken,away,awkward,babi,bachelor,back,bad,balanc,band,bank,base,battl,bear,beauti,becom,befriend,begin,behind,believ,belov,best,betray,better,beyond,bheem,big,biggest,bill,biopic,birth,birthday,black,blind,bodi,bollywood,bond,book,born,bos,boy,boyfriend,break,bring,british,brother,brutal,buddi,build,bulli,busi,businessman,but,call,camp,cancer,captur,car,care,career,carri,cartel,case,cash,cast,cat,catch,caught,caus,celebr,center,centuri,challeng,champion,chanc,chang,chao,charact,charg,charm,chase,chef,child,childhood,choic,choos,christma,chronicl,cia,circumst,citi,civil,claim,clash,class,classic,classmat,client,close,club,coach,collect,colleg,color,combat,come,comedi,comedian,comic,commit,common,commun,compani,compet,competit,complet,complex,complic,con,condit,conflict,confront,connect,conspiraci,contest,continu,control,controversi,convict,cook,cop,cope,corrupt,could,countri,coupl,cours,court,crash,creat,creativ,creatur,crew,crime,crimin,crisi,cross,crush,cultur,dad,danc,dancer,danger,dare,dark,date,daughter,day,de,dead,deadli,deal,dealer,death,debt,decad,decid,deep,...,scienc,scientist,search,season,second,secret,secur,see,seek,seem,seemingli,self,sell,sen,send,senior,sent,separ,sequel,seri,serial,serv,servic,set,seven,sever,sex,sexual,shape,share,shi,shock,shop,short,shot,show,sibl,side,sight,sing,singer,singl,sinist,sister,six,sketch,skill,small,smart,soccer,social,societi,soldier,solv,someth,son,song,soon,soul,sound,south,space,spain,spanish,spark,special,spend,spi,spirit,sport,squad,stage,stand,star,start,state,station,stay,steal,step,still,stolen,stop,stori,strand,strang,stranger,street,strive,struggl,student,stumbl,success,sudden,suddenli,suffer,suicid,summer,super,superhero,supernatur,surpris,surround,surviv,survivor,suspect,suspici,sweet,system,tackl,take,taken,tale,talent,talk,target,task,teach,teacher,team,tech,technolog,teen,teenag,tell,tension,terrifi,terror,terrorist,test,the,thi,thief,thing,think,threat,threaten,three,thriller,throw,thrown,tim,time,to,togeth,top,topic,torn,tough,tour,toward,town,trace,track,trade,tradit,traffick,tragedi,tragic,train,transform,trap,traumat,travel,treasur,tri,trial,triangl,trio,trip,troubl,true,truth,turn,tv,twin,twist,two,unawar,uncov,undercov,underworld,unexpect,unexpectedli,unfold,uniqu,unit,univers,unleash,unlik,unravel,up,upon,upsid,use,vacat,vanish,ve,version,veteran,victim,video,view,villag,villain,violenc,violent,vision,visit,vow,wake,want,war,warrior,water,way,wealthi,web,wed,well,west,when,while,white,whose,widow,wife,wild,will,win,wish,wit,with,without,woman,wonder,work,worker,world,would,write,writer,wrong,year,yet,york,young,younger,youth,g_target_ages
0,the black princ,drama intern movi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.225208,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.294115,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [51]:
test_df_export = X_test_transformed_df_full.drop(['plot_clean'], axis=1).copy()
test_df_export = test_df_export.merge(y_test, on=test_df_export.index)
test_df_export = test_df_export.set_index(['key_0'])
test_df_export.reset_index(drop=True, inplace=True)
#del test_df_export.index.name
test_df_export

,title_clean,list_clean,000,10,11,1960,1970,1980,20,30,abandon,abil,abus,accept,accid,accident,account,accus,across,act,action,activist,actor,actress,adapt,addict,adopt,adult,adventur,affair,africa,african,after,ag,age,agent,ago,agre,air,alien,aliv,all,alli,alon,along,also,alway,ambiti,america,american,amid,among,an,ancient,and,angel,anim,anoth,answer,anyth,apart,appear,arm,armi,around,arrang,arriv,art,artist,ask,aspir,assassin,assist,at,athlet,attack,attempt,attend,attract,author,aveng,avoid,await,awaken,away,awkward,babi,bachelor,back,bad,balanc,band,bank,base,battl,bear,beauti,becom,befriend,begin,behind,believ,belov,best,betray,better,beyond,bheem,big,biggest,bill,biopic,birth,birthday,black,blind,bodi,bollywood,bond,book,born,bos,boy,boyfriend,break,bring,british,brother,brutal,buddi,build,bulli,busi,businessman,but,call,camp,cancer,captur,car,care,career,carri,cartel,case,cash,cast,cat,catch,caught,caus,celebr,center,centuri,challeng,champion,chanc,chang,chao,charact,charg,charm,chase,chef,child,childhood,choic,choos,christma,chronicl,cia,circumst,citi,civil,claim,clash,class,classic,classmat,client,close,club,coach,collect,colleg,color,combat,come,comedi,comedian,comic,commit,common,commun,compani,compet,competit,complet,complex,complic,con,condit,conflict,confront,connect,conspiraci,contest,continu,control,controversi,convict,cook,cop,cope,corrupt,could,countri,coupl,cours,court,crash,creat,creativ,creatur,crew,crime,crimin,crisi,cross,crush,cultur,dad,danc,dancer,danger,dare,dark,date,daughter,day,de,dead,deadli,deal,dealer,death,debt,decad,decid,deep,...,scienc,scientist,search,season,second,secret,secur,see,seek,seem,seemingli,self,sell,sen,send,senior,sent,separ,sequel,seri,serial,serv,servic,set,seven,sever,sex,sexual,shape,share,shi,shock,shop,short,shot,show,sibl,side,sight,sing,singer,singl,sinist,sister,six,sketch,skill,small,smart,soccer,social,societi,soldier,solv,someth,son,song,soon,soul,sound,south,space,spain,spanish,spark,special,spend,spi,spirit,sport,squad,stage,stand,star,start,state,station,stay,steal,step,still,stolen,stop,stori,strand,strang,stranger,street,strive,struggl,student,stumbl,success,sudden,suddenli,suffer,suicid,summer,super,superhero,supernatur,surpris,surround,surviv,survivor,suspect,suspici,sweet,system,tackl,take,taken,tale,talent,talk,target,task,teach,teacher,team,tech,technolog,teen,teenag,tell,tension,terrifi,terror,terrorist,test,the,thi,thief,thing,think,threat,threaten,three,thriller,throw,thrown,tim,time,to,togeth,top,topic,torn,tough,tour,toward,town,trace,track,trade,tradit,traffick,tragedi,tragic,train,transform,trap,traumat,travel,treasur,tri,trial,triangl,trio,trip,troubl,true,truth,turn,tv,twin,twist,two,unawar,uncov,undercov,underworld,unexpect,unexpectedli,unfold,uniqu,unit,univers,unleash,unlik,unravel,up,upon,upsid,use,vacat,vanish,ve,version,veteran,victim,video,view,villag,villain,violenc,violent,vision,visit,vow,wake,want,war,warrior,water,way,wealthi,web,wed,well,west,when,while,white,whose,widow,wife,wild,will,win,wish,wit,with,without,woman,wonder,work,worker,world,would,write,writer,wrong,year,yet,york,young,younger,youth,g_target_ages
0,step sister,comedi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.326766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.262249,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

---
Finally, export our train and test sets, now with over 5000 features, to tsv files. These file are large - ~500MB for train and ~160MB for test.

In [52]:
%%time
train_df_export.to_csv('Task4preprocessed/netflix_train_dataframe.tsv', sep='\t')

CPU times: total: 2.61 s
Wall time: 3.13 s


In [53]:
%%time
test_df_export.to_csv('Task4preprocessed/netflix_test_dataframe.tsv', sep='\t')

CPU times: total: 828 ms
Wall time: 1.07 s
